# Initialize

In [0]:
#!import "../Utils/ImportCalculationMethods"

## DataSource

In [0]:
await DataSource.SetAsync();
DataSource.Reset(x => x.ResetCurrentPartitions());

# Dimensions

## AmountType

In [0]:
var amountTypes = @"@@AmountType
SystemName,DisplayName,Parent
T,Technical Result,
PR,Premium,T
LO,Loss,T
RA,Risk Adjustment,
CI,Claims Incurred,
CL,Claims,T
NIC,Claims Paid (LIC),CL
CE,Claims Expense,CL
E,Expenses,
DAE,Deferred Acquisition Expenses,E
AEM,Maintenance Expenses,E
CE,Claim Expenses,E
O,Other,
CU,CoverageUnits,O
R,Ratios,
LR,LossRatio,R
X,Patterns,
XC,Claims Incurred Pattern,X
XN,Claims Paid Pattern,X
A,AssetClasses,
CB,CouponBond,A
CBC,Coupon,CB
CBF,FaceValue,CB
";

In [0]:
await Import.FromString(amountTypes).WithType<AmountType>().WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Query<AmountType>().Take(3).ToArrayAsync()

## Cashflow Definition
### Basic

In [0]:
Workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var basicCashflowDefinitions = @"@@BasicCashflowDefinition
DataNode,AmountType,Shape,InitialValue,Length,Shift
G.BBA.LRC.2020.P,PR,Constant,10000,5,0
G.BBA.LRC.2023.P,PR,Constant,12000,5,0
G.BBA.LRC.2022.P,CBC,Constant,25,4,1
G.BBA.LRC.2022.P,CBF,Constant,1000,1,4
";

In [0]:
await Import.FromString(basicCashflowDefinitions).WithType<BasicCashflowDefinition>().WithTarget(Workspace).ExecuteAsync()

In [0]:
Workspace.Query<BasicCashflowDefinition>().Take(3)

### Bounded

In [0]:
var boundedCashflowDefinitions = @"@@BoundedCashflowDefinition
DataNode,AmountType,Shape,InitialValue,Length,Shift,Minimum,Maximum
G.BBA.LRC.2019.P,PR,Random,6000,4,1,0,12000
";

In [0]:
await Import.FromString(boundedCashflowDefinitions).WithType<BoundedCashflowDefinition>().WithTarget(Workspace).ExecuteAsync()

In [0]:
Workspace.Query<BoundedCashflowDefinition>().Take(3)

### Referenced

In [0]:
var referencedCashflowDefinition = @"@@ReferencedCashflowDefinition
DataNode,AmountType,AccidentYear,Shape,InitialValue,Length,Shift,ReferenceAmountType,Values0,Values1,Values2,Values3,Values4
G.BBA.LRC.2020.P,CL,,Reference,,,,PR,0.75,0.75,0.75,0.75,0.75
G.BBA.LRC.2020.P,DAE,,Reference,,,,PR,0.2,0.2,0.2,0.2,0.2
G.BBA.LRC.2020.P,AEM,,Reference,,,,PR,0.02,0.02,0.02,0.02,0.02
G.BBA.LRC.2020.P,CE,,Reference,,,,NIC,,,,,
G.BBA.LRC.2023.P,CE,,Reference,,,,NIC,-0.03,-0.03,-0.03,-0.03,-0.03
G.BBA.LRC.2020.P,RA,,Reference,,,,CL,0.04,0.04,0.04,0.04,0.04
G.BBA.LRC.2023.P,RA,2023,Reference,,,,CL,0.04,0.04,0.04,0.04,0.04
G.BBA.LRC.2023.P,RA,2024,Reference,,,1,CL,0.04,0.04,0.04,0.04,0.04
G.BBA.LRC.2023.P,RA,2025,Reference,,,2,CL,0.04,0.04,0.04,0.04,0.04
G.BBA.LRC.2023.P,RA,2026,Reference,,,3,CL,0.04,0.04,0.04,0.04,0.04
G.BBA.LRC.2023.P,RA,2027,Reference,,,4,CL,0.04,0.04,0.04,0.04,0.04
G.BBA.LRC.2023.P,RA,2028,Reference,,,5,CL,0.04,0.04,0.04,0.04,0.04
";

In [0]:
await Import.FromString(referencedCashflowDefinition).WithType<ReferencedCashflowDefinition>().WithTarget(Workspace).ExecuteAsync()

In [0]:
Workspace.Query<ReferencedCashflowDefinition>().Take(3)